Multi-layer perceptron classifier

In [ ]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn funcitonal as F 
import torch.backends.cuda as cudnn 

import torchvision 
import torchvision.transofrms as transoforms 

import matplotlib.pyplot as plt 
import numpy as np 

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


trainset = torchvision.datasets.CIFAR10(root='../data', train= True, download=True, transform=transform)
trainloader = torchutils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvisoin.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)


In [ ]:
def imshow(img):
    plt.figure(figsize=(10, 10))
    img = img / 2 + 0.5 #unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()


imshow(torchvision.utils.make_grid(images))

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, n_channels, n_output_neurons):
        super().__init__()

        self.main = nn.Sequential( #this is the convolutional part
            nn.Conv2d(n_channels, 16, kernel_size=5, stride=2, padding=0),
            nn.ReLU(inplace=True)
            nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=0),
            nn.ReLU(inplace=True)
            nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=0),
            nn.ReLU(inplace=True)
            

        )


        self.mlp = nn.Sequential( # this is the multi layer perceptron part
            nn.Linear(64, 32),
            nn.LeakyReLU(inplace=True),
            nn.Linear(32, n_output_neurons)

        )


        def forward(self, x):
            batch_size = x.shape[0]

            x = self.main(x)

            x = x.view(batch_size, -1)

            return self.mlp(x)
    

In [ ]:
height, width = images.shape[2:]

In [ ]:
images.shape

In [ ]:
net = ConvNet(n_channels=3, n_output_neurons= 10).cuda()

In [ ]:
net(images.cuda()).shape

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(5): #loop over the dataset multiple times

    running_loss = 0.0 
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        images, labels = data
        images, labels = images.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        # print statistics
        running_loss += loss.items()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(' [%d, %5d] loss: %.3f' %
             (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
def get_quality(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader: 
            images, labels = data 
            images, labels = images.cuda(), labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print('Accuracy of the network on test images: %d %%' % (100 * correct / total))

In [ ]:
get_quality(net)